In [1]:
import sys
sys.path.insert(0, "/work/04233/sw33286/AIDA-SCRIPTS")
sys.path.insert(0, "/home/04233/sw33286/AIDA-package")

In [2]:
import os
import time
import random
import shutil
import dill
import numpy as np

import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell, DropoutWrapper

from kmedoids import kMedoids
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import accuracy_score

from helpers import Indexer, batch
from itertools import chain, product
from collections import defaultdict

### Prepare data

In [3]:
# Link to NYT data folder

nyt_code_dir = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_event_code/"
FILE_NAMES = os.listdir(nyt_code_dir)

# Link to dictionary information

info_path = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_info/indexer_word2emb_100k_event.p"
indexer100k, word2emb100k = dill.load(open(info_path, 'rb'))
glove_embs = []
for i in range(len(indexer100k)):
    glove_embs.append(word2emb100k[indexer100k.get_object(i)])
glove_embs = np.array(glove_embs)
print(glove_embs.shape)

(77277, 300)


In [4]:
BATCH_SIZE = 32
NUM_EVENTS = 5
NUM_WORDS = 5
CTX_LEN = 20
CTX_DUMMY = np.zeros([NUM_EVENTS, NUM_WORDS])

def get_batch(file_idx):
    filename = FILE_NAMES[file_idx]
    edoc_a, edoc_b, batch_ctx = dill.load(open(nyt_code_dir+FILE_NAMES[file_idx],'rb'))
    size_a, size_b, size_ctx = len(edoc_a), len(edoc_b), len(batch_ctx)
    batch_x1, batch_x2, batch_y = [], [], []
    ys = [1,0,0,1]
    sizes = [[size_a,size_a],[size_a,size_b],[size_b,size_a],[size_b,size_b]]
    for _ in range(BATCH_SIZE//4):
        for i,(eda,edb) in enumerate(product([edoc_a,edoc_b],
                                             [edoc_a,edoc_b])):
            batch_x1.append(eda[np.random.randint(0,sizes[i][0])])
            batch_x2.append(edb[np.random.randint(0,sizes[i][1])])
            batch_y.append(ys[i])
    batch_ctx = batch_ctx[:CTX_LEN] if size_ctx>=CTX_LEN else batch_ctx+[CTX_DUMMY]*(CTX_LEN-size_ctx)
    return np.array(batch_x1), np.array(batch_x2), np.array(batch_ctx), np.array(batch_y) 
        # batch_x*: <bc,ne,nw>, batch_ctx batch_y: <bc,>

In [5]:
# Example: batch shapes

a1,a2,b,c = get_batch(0)
a1.shape, a2.shape, b.shape, c.shape

((32, 5, 5), (32, 5, 5), (20, 5, 5), (32,))

### FFNN-BiLSTM-bilinear

In [20]:
tf.reset_default_graph()

sess = tf.InteractiveSession()

VOCAB_SIZE, EMB_SIZE = glove_embs.shape
HID_SIZE = 100 # let event embs be of the same hid-size as role-factored arg vectors.

NUM_LAYERS = 2
KEEP_PROB = 0.7
LEARNING_RATE = 1e-5

# hyperparams for cnn context reader.
FILTER_SIZES = [3,4,5]
NUM_FILTERS = 50
NUM_CHANNELS = 1

input_x1 = tf.placeholder(tf.int32, [None, NUM_EVENTS, NUM_WORDS], name='input_x1')
input_x2 = tf.placeholder(tf.int32, [None, NUM_EVENTS, NUM_WORDS], name='input_x2')
input_ctx = tf.placeholder(tf.int32, [CTX_LEN, NUM_EVENTS, NUM_WORDS], name='input_ctx')
input_y = tf.placeholder(tf.int32, [None], name='input_y')

keep_prob = tf.placeholder(tf.float32, name="keep_prob")

seq_lens = tf.placeholder(tf.int32, [None], name='seq_lens')
bc,_,_ = tf.unstack(tf.shape(input_x1))

with tf.variable_scope('Embedding'):
    embeddings = tf.get_variable('embedding', [VOCAB_SIZE, EMB_SIZE],
                                 initializer=tf.contrib.layers.xavier_initializer())
    glove_init = embeddings.assign(glove_embs)
    input_x1_embedded = tf.reshape(tf.nn.embedding_lookup(embeddings, input_x1), [bc, NUM_EVENTS, -1]) 
        # op1. embed words: <bc,ne,nw,emb>
        # op2. concat words in event: <bc,ne,nw*emb>
    input_x2_embedded = tf.reshape(tf.nn.embedding_lookup(embeddings, input_x2), [bc, NUM_EVENTS, -1])
    input_ctx_embedded = tf.reshape(tf.nn.embedding_lookup(embeddings, input_ctx), [CTX_LEN, NUM_EVENTS, -1])
        # op1. embed words: <ctx,ne,nw,emb>
        # op2. concat words in event: <ctx,ne,nw*emb>

def run_ffnn(inputs): # inputs=<ne,nw*emb>
    W_ffnn = tf.get_variable('W_ffnn', [NUM_WORDS*EMB_SIZE, EMB_SIZE], 
                             initializer=tf.contrib.layers.xavier_initializer())
    return tf.matmul(inputs, W_ffnn) # <ne,emb>, event now has the same emb length as word.

with tf.variable_scope('FFNN') as scope:
    input_x1_ffnn = tf.transpose(tf.map_fn(run_ffnn, input_x1_embedded), [1,0,2]) 
        # op1. map_fn out: <bc,ne,emb>
        # op2. for lstm input: <max-time=ne,bc,emb>
    scope.reuse_variables()
    input_x2_ffnn = tf.transpose(tf.map_fn(run_ffnn, input_x2_embedded), [1,0,2])
    scope.reuse_variables()
    input_ctx_ffnn = tf.expand_dims(tf.expand_dims(tf.reshape(tf.map_fn(run_ffnn, input_ctx_embedded),
                                                              [CTX_LEN,-1]),0),-1) 
        # op1. lookup: <ctx,ne,emb>
        # op2. concat events in context: <ctx,ne*emb>
        # op2. expand dim for bc=1: <bc=1,height=ctx,width=ne*emb>
        # op3. expand dim for chn=1: <bc=1,height=ctx,width=ne*emb,chn=1>
    
cell = MultiRNNCell([DropoutWrapper(LSTMCell(HID_SIZE),output_keep_prob=keep_prob)]*NUM_LAYERS)

def run_lstm(inputs):
    ((fw_outputs,bw_outputs), # <max-time=ne,bc,hid>, attention later if needed.
     (fw_final_state,bw_final_state)) = ( # <bc,hid>
        tf.nn.bidirectional_dynamic_rnn(cell_fw=cell,
                                        cell_bw=cell,
                                        inputs=inputs,
                                        sequence_length=seq_lens,
                                        dtype=tf.float32, time_major=True)
    )    
    return tf.concat([tf.concat([fw_state_tuple.h,bw_state_tuple.h], 1) # lstm-out size *= NUM_LAYERS by stacking.
                      for fw_state_tuple,bw_state_tuple in zip(fw_final_state,bw_final_state)], 1), \
           tf.transpose(tf.concat([fw_outputs,bw_outputs], 2), [1,0,2])
        # out1: <bc,hid*2*num-layers>
        # out2: concat -> <ne,bc,hid*2> -> <bc,ne,hid*2>

with tf.variable_scope('BiLSTM') as scope: 
    final_state_x1, outputs_x1 = run_lstm(input_x1_ffnn)
        # fs_x1: <bc,hid*2*num-layers>
        # out_x1: <bc,ne,hid*2>
    scope.reuse_variables()
    final_state_x2, outputs_x2 = run_lstm(input_x2_ffnn)
    
def run_attention(outputs, state):
    W_d = tf.get_variable('W_d', [HID_SIZE*2, HID_SIZE*2], initializer=tf.contrib.layers.xavier_initializer())
    W_s = tf.get_variable('W_s', [HID_SIZE*2*NUM_LAYERS, HID_SIZE*2], 
                          initializer=tf.contrib.layers.xavier_initializer())
    d_W = tf.tensordot(outputs, W_d, axes=[[2],[0]])
        # <bc,ne,hid*2> * <hid*2,hid*2> = <bc,ne,hid*2>
    s_W = tf.expand_dims(tf.matmul(state, W_s), axis=1)
        # op1. <bc,hid*2*num-layers> * <hid*2*num-layers,hid*2> -> <bc,hid*2>
        # op2. <bc,hid*2> -> <bc,1,hid*2>
    a_tsr = tf.nn.tanh(tf.add(d_W, s_W))
        # op1. <bc,ne,hid*2> + <bc,1,hid*2> -> <bc,ne,hid*2>
        # op2. elem-wise nonlinearity.
    W_a = tf.get_variable('W_a', [HID_SIZE*2, 1], initializer=tf.contrib.layers.xavier_initializer())
    a_W = tf.nn.softmax(tf.tensordot(a_tsr, W_a, axes=[[2],[0]]), dim=1)
        # op1. <bc,ne,hid*2> * <hid*2,1> -> <bc,ne,1>
        # op2. softmax over max-time=ne.
    d_a = tf.reduce_sum(tf.multiply(outputs, a_W), axis=1)
        # op1. <bc,ne,hid*2> elem* <bc,ne,1> -> <bc,ne,hid*2>
        # op2. sum over max-time=ne (weighted sum) -> <bc,hid*2>
    return d_a  

with tf.variable_scope('Mutual-Attention') as scope:
    x1_to_x2_att = run_attention(outputs_x2, final_state_x1) # x1 attending to x2, <bc,hid*2>
    scope.reuse_variables()
    x2_to_x1_att = run_attention(outputs_x1, final_state_x2) # x2 attending to x1
    
def run_cnn(inputs): # in: <bc=1,height=ctx,width=ne*emb,chn=1>
    pool_outputs = []
    for i,filter_size in enumerate(FILTER_SIZES):
        with tf.variable_scope('CNN-ctx-%s' % filter_size):
            filter_shape = [filter_size, NUM_EVENTS*EMB_SIZE, NUM_CHANNELS, NUM_FILTERS]
            W = tf.get_variable('W', filter_shape, initializer=tf.contrib.layers.xavier_initializer())
            b = tf.get_variable('b', [NUM_FILTERS], initializer=tf.contrib.layers.xavier_initializer())
            conv = tf.nn.conv2d(inputs, W, strides=[1,1,1,1], padding='VALID', name='conv')
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name='relu')
            pool = tf.nn.max_pool(h, ksize=[1,CTX_LEN-filter_size+1,1,1], strides=[1,1,1,1], padding='VALID', name='pool')
            pool_outputs.append(pool)
    num_filters_total = NUM_FILTERS * len(FILTER_SIZES)
    h_pool_flat = tf.nn.dropout(tf.reshape(tf.concat(pool_outputs, 3), [-1, num_filters_total]), keep_prob)
    return h_pool_flat # <bc=1,num-filters*len(filter-sizes)>

with tf.variable_scope('Context-reader') as scope:
    ctx = tf.tile(run_cnn(input_ctx_ffnn), [bc,1])
        # op1. run-cnn out: <bc=1,num-filters*len(filter-sizes)>
        # op2. create bc copies of it: <bc,num-filters*len(filter-sizes)>
        
def run_scores(fs_x1, fs_x2, att_12, att_21, c):
    fv_size = HID_SIZE*2*NUM_LAYERS+HID_SIZE*2+NUM_FILTERS*len(FILTER_SIZES) 
        # sent encoding size + mutual attention size + context size.
        # e.g. hid-size=100, num-filters=50
        #      400 + 200 + 150 = 750
    W_bi = tf.get_variable('W_bi', [fv_size, fv_size], 
                           initializer=tf.contrib.layers.xavier_initializer())
    fv_x1 = tf.concat([fs_x1,att_12,c],axis=1) 
        # concat: [<bc,hid*2*num-layers>, <bc,hid*2>, <bc,num-filters*len(filter-sizes)>]
        #   -> <bc,fv = hid*2*num-layers + hid*2 + num-filters*len(filter-sizes)>
    fv_x2 = tf.concat([fs_x2,att_21,c],axis=1)
    return tf.nn.sigmoid(tf.diag_part(tf.matmul(tf.matmul(fv_x1,W_bi),tf.transpose(fv_x2))),name='scores')
        # op1. bilinear mult: <bc,fv> * <fv,fv> * <fv,bc> -> <bc,bc>
        # op2: match bc: <bc,>
        # op3: sigmoid to compute scores: <bc,>
        
scores = run_scores(final_state_x1, final_state_x2, x1_to_x2_att, x2_to_x1_att, ctx)
predictions = tf.cast(tf.round(scores), tf.int32, name='predictions')     
    
with tf.name_scope('Loss'):
    losses = tf.nn.sigmoid_cross_entropy_with_logits(labels=tf.cast(input_y, tf.float32), logits=scores)
    loss = tf.reduce_mean(losses, name='loss')

with tf.name_scope('Accuracy'):
    correct_predictions = tf.equal(predictions, input_y)
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name='accuracy')
    
global_step = tf.Variable(0, name='global_step', trainable=False)
optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
grads_and_vars = optimizer.compute_gradients(loss)
train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step, name='train_op')

sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()
save_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE-EVENT/our-model-ffnn-bilstm/"
save_path = save_dir + "our-model-ffnn-bilstm-00"

def remove_all_files(target_dir):
    for filename in os.listdir(target_dir):
        os.remove(os.path.abspath(os.path.join(target_dir, filename)))
        
def checkpoint_model(s_dir, s_path, svr, ss):
    # save_dir, save_path, saver, sess
    remove_all_files(s_dir)
    svr.save(ss, s_path)

In [52]:
NUM_EPOCHS = 1
TRAIN_SIZE = 10
VERBOSE = 1
SAVE_PER = 50000
# TRAIN_SIZE = len(FILE_NAMES)
# VERBOSE = 1000

loss_track, accuracy_track = [], []
start = time.time()
try:
    for e in range(NUM_EPOCHS):
        print('Epoch ', e+1)
        print('\n')
        file_indices = np.random.choice(list(range(len(FILE_NAMES))), size=TRAIN_SIZE, replace=False)
        random.shuffle(file_indices)
        curr_loss_track, curr_accuracy_track = [], []
        for file_idx in file_indices:
            batch_x1, batch_x2, batch_ctx, batch_y = get_batch(file_idx)
            fd = {input_x1:batch_x1, input_x2:batch_x2,
                  input_ctx:batch_ctx,
                  input_y:batch_y,
                  keep_prob:KEEP_PROB,
                  seq_lens:[NUM_EVENTS]*batch_x1.shape[0]}
            _, step, loss_, accuracy_ = sess.run([train_op, global_step, loss, accuracy], feed_dict=fd)
            curr_loss_track.append(loss_)
            curr_accuracy_track.append(accuracy_)
            if step%VERBOSE==0:
                print(' average batch loss & accuracy at step {}: <{}, {}> (time elapsed = {})'.format(step,
                                                                                   np.mean(curr_loss_track), 
                                                                                   np.mean(curr_accuracy_track),
                                                                                   time.time()-start))
                start = time.time()
            if step%SAVE_PER==0:
                checkpoint_model(save_dir, save_path, saver, sess)
                print(' [SAVE] model checkpointed at step {}'.format(step))
    print('\n')
    print('  epoch mean loss & accuracy: <{}, {}>'.format(np.mean(curr_loss_track),np.mean(curr_accuracy_track)))
    print('\n') 
    loss_track += curr_loss_track
    accuracy_track += curr_accuracy_track
except KeyboardInterrupt:
    print('Stopped!')            

### Model saving

In [23]:
checkpoint_model(save_dir, save_path, saver, sess)

### Model continued training

In [24]:
import sys
sys.path.insert(0, "/work/04233/sw33286/AIDA-SCRIPTS")
sys.path.insert(0, "/home/04233/sw33286/AIDA-package")

import os
import time
import random
import shutil
import dill
import numpy as np

import tensorflow as tf
from tensorflow.contrib.rnn import LSTMCell, MultiRNNCell, DropoutWrapper

from kmedoids import kMedoids
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import accuracy_score

from helpers import Indexer, batch
from itertools import chain, product
from collections import defaultdict

# Link to NYT data folder

nyt_code_dir = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_event_code/"
FILE_NAMES = os.listdir(nyt_code_dir)

# Link to dictionary information

info_path = "/work/04233/sw33286/AIDA-DATA/nyt_eng_salads_info/indexer_word2emb_100k_event.p"
indexer100k, word2emb100k = dill.load(open(info_path, 'rb'))
glove_embs = []
for i in range(len(indexer100k)):
    glove_embs.append(word2emb100k[indexer100k.get_object(i)])
glove_embs = np.array(glove_embs)
print(glove_embs.shape)

BATCH_SIZE = 32
NUM_EVENTS = 5
NUM_WORDS = 5
CTX_LEN = 20
CTX_DUMMY = np.zeros([NUM_EVENTS, NUM_WORDS])

def get_batch(file_idx):
    filename = FILE_NAMES[file_idx]
    edoc_a, edoc_b, batch_ctx = dill.load(open(nyt_code_dir+FILE_NAMES[file_idx],'rb'))
    size_a, size_b, size_ctx = len(edoc_a), len(edoc_b), len(batch_ctx)
    batch_x1, batch_x2, batch_y = [], [], []
    ys = [1,0,0,1]
    sizes = [[size_a,size_a],[size_a,size_b],[size_b,size_a],[size_b,size_b]]
    for _ in range(BATCH_SIZE//4):
        for i,(eda,edb) in enumerate(product([edoc_a,edoc_b],
                                             [edoc_a,edoc_b])):
            batch_x1.append(eda[np.random.randint(0,sizes[i][0])])
            batch_x2.append(edb[np.random.randint(0,sizes[i][1])])
            batch_y.append(ys[i])
    batch_ctx = batch_ctx[:CTX_LEN] if size_ctx>=CTX_LEN else batch_ctx+[CTX_DUMMY]*(CTX_LEN-size_ctx)
    return np.array(batch_x1), np.array(batch_x2), np.array(batch_ctx), np.array(batch_y) 
        # batch_x*: <bc,ne,nw>, batch_ctx batch_y: <bc,>

(77277, 300)


In [25]:
restore_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE-EVENT/our-model-ffnn-bilstm/"
restore_filename = "our-model-ffnn-bilstm-00.meta"

sess = tf.Session()
saver = tf.train.import_meta_graph(restore_dir + restore_filename)
saver.restore(sess, tf.train.latest_checkpoint(restore_dir))
graph = tf.get_default_graph()

input_x1 = graph.get_tensor_by_name('input_x1:0')
input_x2 = graph.get_tensor_by_name('input_x2:0')
input_ctx = graph.get_tensor_by_name('input_ctx:0')
input_y = graph.get_tensor_by_name('input_y:0')
keep_prob = graph.get_tensor_by_name('keep_prob:0')
seq_lens = graph.get_tensor_by_name('seq_lens:0')

scores = graph.get_tensor_by_name('scores:0')
predictions = graph.get_tensor_by_name('predictions:0')
loss = graph.get_tensor_by_name('Loss/loss:0')
accuracy = graph.get_tensor_by_name('Accuracy/accuracy:0')
global_step = graph.get_tensor_by_name('global_step:0')
train_op = graph.get_tensor_by_name('train_op:0')

save_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE-EVENT/our-model-ffnn-bilstm/"
save_path = save_dir + "our-model-ffnn-bilstm-00"

def remove_all_files(target_dir):
    for filename in os.listdir(target_dir):
        os.remove(os.path.abspath(os.path.join(target_dir, filename)))
        
def checkpoint_model(s_dir, s_path, svr, ss):
    # save_dir, save_path, saver, sess
    remove_all_files(s_dir)
    svr.save(ss, s_path)

In [53]:
NUM_EPOCHS = 1
TRAIN_SIZE = 10
VERBOSE = 1
SAVE_PER = 50000
# TRAIN_SIZE = len(FILE_NAMES)
# VERBOSE = 1000

loss_track, accuracy_track = [], []
start = time.time()
try:
    for e in range(NUM_EPOCHS):
        print('Epoch ', e+1)
        print('\n')
        file_indices = np.random.choice(list(range(len(FILE_NAMES))), size=TRAIN_SIZE, replace=False)
        random.shuffle(file_indices)
        curr_loss_track, curr_accuracy_track = [], []
        for file_idx in file_indices:
            batch_x1, batch_x2, batch_ctx, batch_y = get_batch(file_idx)
            fd = {input_x1:batch_x1, input_x2:batch_x2,
                  input_ctx:batch_ctx,
                  input_y:batch_y,
                  keep_prob:KEEP_PROB,
                  seq_lens:[NUM_EVENTS]*batch_x1.shape[0]}
            _, step, loss_, accuracy_ = sess.run([train_op, global_step, loss, accuracy], feed_dict=fd)
            curr_loss_track.append(loss_)
            curr_accuracy_track.append(accuracy_)
            if step%VERBOSE==0:
                print(' average batch loss & accuracy at step {}: <{}, {}> (time elapsed = {})'.format(step,
                                                                                   np.mean(curr_loss_track), 
                                                                                   np.mean(curr_accuracy_track),
                                                                                   time.time()-start))
                start = time.time()
            if step%SAVE_PER==0:
                checkpoint_model(save_dir, save_path, saver, sess)
                print(' [SAVE] model checkpointed at step {}'.format(step))
    print('\n')
    print('  epoch mean loss & accuracy: <{}, {}>'.format(np.mean(curr_loss_track),np.mean(curr_accuracy_track)))
    print('\n') 
    loss_track += curr_loss_track
    accuracy_track += curr_accuracy_track
except KeyboardInterrupt:
    print('Stopped!')            

In [27]:
# Final Save
        
checkpoint_model(save_dir, save_path, saver, sess)

### K-Medoids evaluation

In [49]:
# Evaluation functions

def get_rand_mixture():
    filename = random.choice(FILE_NAMES)
    da,db, doc_mix = dill.load(open(nyt_code_dir+filename, 'rb'))
    doc_lbs = []
    for sentcode in doc_mix:
        if sentcode in da:
            doc_lbs.append(0)
        else:
            doc_lbs.append(1)
    size_ctx = len(doc_mix)
    ctx = doc_mix[:CTX_LEN] if size_ctx>=CTX_LEN else np.vstack((doc_mix,[CTX_DUMMY]*(CTX_LEN-size_ctx)))
    return np.reshape(np.array(doc_mix),[-1,NUM_EVENTS*NUM_WORDS]), np.array(doc_lbs), ctx

def to_labels(C, doc_len): # C: {cls:[datum_id, ...], ...}
    lbs = [0]*doc_len
    for idx in C[1]:
        lbs[idx] = 1
    return lbs

def flip_clust(clust):
    return np.array([0 if i==1 else 1 for i in clust])

def clust_accuracy(true, pred):
    return max(accuracy_score(true, pred),
               accuracy_score(true, flip_clust(pred)))

def to_sent(code):
    return [indexer100k.get_object(idx) for idx in code]

# Bi-LSTM + HAC class

class ClfKM:
    
    def __init__(self, clf_dir, clf_filename):
        self.sess = tf.Session()
        saver = tf.train.import_meta_graph(clf_dir + clf_filename)
        saver.restore(self.sess, tf.train.latest_checkpoint(clf_dir))
        self.graph = tf.get_default_graph()
        self.input_x1 = self.graph.get_tensor_by_name('input_x1:0')
        self.input_x2 = self.graph.get_tensor_by_name('input_x2:0')
        self.input_ctx = self.graph.get_tensor_by_name('input_ctx:0')
        self.input_y = self.graph.get_tensor_by_name('input_y:0')
        self.keep_prob = self.graph.get_tensor_by_name('keep_prob:0')
        self.seq_lens = self.graph.get_tensor_by_name('seq_lens:0')

        self.scores = self.graph.get_tensor_by_name('scores:0')
        self.predictions = self.graph.get_tensor_by_name('predictions:0')
        self.loss = self.graph.get_tensor_by_name('Loss/loss:0')
        self.accuracy = self.graph.get_tensor_by_name('Accuracy/accuracy:0')
        self.global_step = self.graph.get_tensor_by_name('global_step:0')
        self.train_op = self.graph.get_tensor_by_name('train_op:0')
        
    def dist(self, x1, x2):
        x1 = np.reshape(x1, [-1, NUM_EVENTS, NUM_WORDS])
        x2 = np.reshape(x2, [-1, NUM_EVENTS, NUM_WORDS])
        fd = {input_x1:x1, input_x2:x2, 
              input_ctx:self.ctx,
              keep_prob:1.0,
              seq_lens:[NUM_EVENTS]}
        conf = self.sess.run(self.scores, feed_dict=fd)
        return 1-conf[0]
    
    def evaluate(self, doc_mix, doc_lbs, ctx, method='average', plot=True):
        self.ctx = ctx
        _, doc_mix_clust = kMedoids(squareform(pdist(doc_mix,metric=self.dist)), 2)
        doc_prd = to_labels(doc_mix_clust, len(doc_mix))
        acc = clust_accuracy(doc_lbs, doc_prd)
        return acc
        

In [50]:
restore_dir = "/work/04233/sw33286/AIDA-INDIV-MODEL-SAVE-EVENT/our-model-ffnn-bilstm/"
restore_filename = "our-model-ffnn-bilstm-00.meta"
clf_km = ClfKM(restore_dir, restore_filename)

In [54]:
clf_km.evaluate(*get_rand_mixture())

### Evaluation on subset

In [ ]:
def rand_evaluation(k=100):
    accuracies = []
    for _ in range(k):
        _, acc = clf_km.evaluate(*get_rand_mixture(), plot=False)
        accuracies.append(acc)
    print('Average clustering accuracy over {} samples = {}'.format(k, np.mean(accuracies)))

In [ ]:
%%time

rand_evaluation()